# Taguchi Array

In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from utils import validate_parameters, score_vals, efficiency_calc

In [2]:
MODULE = 0.8e-3
MC = 0.8467e-3

In [3]:
# New input array
# in_vals = [z_s, z_r2, x_s, x_r2, Cl, gear_thickness] # Cl_i = clearance (i = 1,2,3)
param_vals = {
    'z_sh' : [9, 8, 7][::-1],
    'z_r2' : [83, 82, 80][::-1],
    'x_s' : [-0.5, 0, 1],
    'x_r2' : [-0.5, 0, 0.5],
    'Cl' : [0.4e-3, 0.3e-3, 0.2e-3][::-1], #NOTE - use 0.1 for paper, though they likely use 0.0
    'gear_thickness' : [12, 8, 6][::-1],
}

# paper_vals = [6, 81, .476, 1.21, 0.1e-3, 10] # NOTE - 6 = 12 (z_s) / 2.
paper_vals = [6, 81, .5, 1, 0.1e-3, 10] # NOTE - 6 = 12 (z_s) / 2.
quick_param = []

for i in range(len(param_vals)):
    key_i = list(param_vals.keys())[i]
    param_vals[key_i].insert(0, paper_vals[i])
    quick_param.append(param_vals[key_i])

headers = list(param_vals.keys())
rows = list(zip(*param_vals.values()))

print(tabulate(rows, headers=headers, tablefmt="grid"))

design_vals, I1, I2, gr_s, ratios = validate_parameters(quick_param[:-1])

design_vals = np.array(design_vals)

print(f'Design vals shape: {design_vals.shape}')

# Print the unique values
cols = ['z_sh', 'z_p1', 'z_p2', 'z_r2', 'xs', 'xp1', 'xp2', 'xr2', 'r_a']
# for i in range(len(cols)):
#     print(f"{cols[i]}: {np.unique(design_vals[i])}")

+--------+--------+-------+--------+--------+------------------+
|   z_sh |   z_r2 |   x_s |   x_r2 |     Cl |   gear_thickness |
+========+========+=======+========+========+==================+
|      6 |     81 |   0.5 |    1   | 0.0001 |               10 |
+--------+--------+-------+--------+--------+------------------+
|      7 |     80 |  -0.5 |   -0.5 | 0.0002 |                6 |
+--------+--------+-------+--------+--------+------------------+
|      8 |     82 |   0   |    0   | 0.0003 |                8 |
+--------+--------+-------+--------+--------+------------------+
|      9 |     83 |   1   |    0.5 | 0.0004 |               12 |
+--------+--------+-------+--------+--------+------------------+
In-vector mesh shape: (4, 4, 4, 4, 4)
In-vector mesh len: 1024
Design vals shape: (9, 1024)


In [4]:
i = 0
quick_param = list(np.array(list(param_vals.values()))[:,i])
print(quick_param)
score = score_vals(quick_param[:-1], add_gear= False)

[np.float64(6.0), np.float64(81.0), np.float64(0.5), np.float64(1.0), np.float64(0.0001), np.float64(10.0)]
In-vector mesh shape: (1, 1, 1, 1, 1)
In-vector mesh len: 1
(9, 1)
Efficiencies: Front - [0.08765368], Back - [0.]
Score compare: best - 1, current - 0


In [5]:
score

0

In [6]:
ratios[4]

np.float64(90.84375)

In [7]:
design_dict = {}
for i in range(len(cols)):
    design_dict[cols[i]] = [design_vals[i,4]]

headers = list(design_dict.keys())
rows = list(zip(*design_dict.values()))


print(tabulate(rows, headers=headers, tablefmt="grid"))

+--------+--------+--------+--------+------+-------+-----------+-------+--------+
|   z_sh |   z_p1 |   z_p2 |   z_r2 |   xs |   xp1 |       xp2 |   xr2 |    r_a |
+========+========+========+========+======+=======+===========+=======+========+
|      6 |     38 |     31 |     81 |  0.5 | 0.125 | 0.0521436 |  -0.5 | 0.0206 |
+--------+--------+--------+--------+------+-------+-----------+-------+--------+


In [8]:
Taguchi = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
[1, 1, 2, 2, 2, 2, 2, 2, 2, 2],
[1, 1, 3, 3, 3, 3, 3, 3, 3, 3],
[1, 1, 4, 4, 4, 4, 4, 4, 4, 4],
[1, 2, 1, 1, 2, 2, 3, 3, 4, 4],
[1, 2, 2, 2, 1, 1, 4, 4, 3, 3],
[1, 2, 3, 3, 4, 4, 1, 1, 2, 2],
[1, 2, 4, 4, 3, 3, 2, 2, 1, 1],
[1, 3, 1, 2, 3, 4, 1, 2, 3, 4],
[1, 3, 2, 1, 4, 3, 2, 1, 4, 3],
[1, 3, 3, 4, 1, 2, 3, 4, 1, 2],
[1, 3, 4, 3, 2, 1, 4, 3, 2, 1],
[1, 4, 1, 2, 4, 3, 3, 4, 2, 1],
[1, 4, 2, 1, 3, 4, 4, 3, 1, 2],
[1, 4, 3, 4, 2, 1, 1, 2, 4, 3],
[1, 4, 4, 3, 1, 2, 2, 1, 3, 4],
[2, 1, 1, 4, 1, 4, 2, 3, 2, 3],
[2, 1, 2, 3, 2, 3, 1, 4, 1, 4],
[2, 1, 3, 2, 3, 2, 4, 1, 4, 1],
[2, 1, 4, 1, 4, 1, 3, 2, 3, 2],
[2, 2, 1, 4, 2, 3, 4, 1, 3, 2],
[2, 2, 2, 3, 1, 4, 3, 2, 4, 1],
[2, 2, 3, 2, 4, 1, 2, 3, 1, 4],
[2, 2, 4, 1, 3, 2, 1, 4, 2, 3],
[2, 3, 1, 3, 3, 1, 2, 4, 4, 2],
[2, 3, 2, 4, 4, 2, 1, 3, 3, 1],
[2, 3, 3, 1, 1, 3, 4, 2, 2, 4],
[2, 3, 4, 2, 2, 4, 3, 1, 1, 3],
[2, 4, 1, 3, 4, 2, 4, 2, 1, 3],
[2, 4, 2, 4, 3, 1, 3, 1, 2, 4],
[2, 4, 3, 1, 2, 4, 2, 4, 3, 1],
[2, 4, 4, 2, 1, 3, 1, 3, 4, 2],
])-1

In [9]:
first_set = []
Tag_vals = Taguchi[0][1:-1]
print(Tag_vals)
for i in range(len(param_vals)):
    first_set.append(param_vals[i][Tag_vals[i]])

[0 0 0 0 0 0 0 0]


KeyError: 0

In [11]:
first_set

[12, 32, 81, 0.476, 0.762, 0.536, 1.21, 10]

In [12]:
targ_set = 2 # 1-32
new_set = []
Tag_vals = Taguchi[targ_set-1][1:-1]
# print(Tag_vals)
for i in range(len(param_vals)):
    new_set.append(param_vals[i][Tag_vals[i]])
new_set

[12, 31, 83, 0.3, 0.6, -0.4, 0.4, 6]

In [13]:
set_list = []
for targ_set in range(len(Taguchi)):
    new_set = []
    Tag_vals = Taguchi[targ_set][1:-1]
    # print(Tag_vals)
    for i in range(len(param_vals)):
        new_set.append(param_vals[i][Tag_vals[i]])
    
    # Print the new set of parameters
    print(f"Set {targ_set + 1}: {new_set}")
    set_list.append(new_set)
    
    # Here you can add code to perform calculations or simulations with new_set if needed

Set 1: [12, 32, 81, 0.476, 0.762, 0.536, 1.21, 10]
Set 2: [12, 31, 83, 0.3, 0.6, -0.4, 0.4, 6]
Set 3: [12, 30, 82, 0.6, 0.9, -0.3, 0.5, 8]
Set 4: [12, 33, 80, 0.7, 1.0, -0.5, 0.3, 12]
Set 5: [18, 32, 81, 0.3, 0.6, -0.3, 0.5, 12]
Set 6: [18, 31, 83, 0.476, 0.762, -0.5, 0.3, 8]
Set 7: [18, 30, 82, 0.7, 1.0, 0.536, 1.21, 6]
Set 8: [18, 33, 80, 0.6, 0.9, -0.4, 0.4, 10]
Set 9: [16, 32, 83, 0.6, 1.0, 0.536, 0.4, 8]
Set 10: [16, 31, 81, 0.7, 0.9, -0.4, 1.21, 12]
Set 11: [16, 30, 80, 0.476, 0.6, -0.3, 0.3, 10]
Set 12: [16, 33, 82, 0.3, 0.762, -0.5, 0.5, 6]
Set 13: [14, 32, 83, 0.7, 0.9, -0.3, 0.3, 6]
Set 14: [14, 31, 81, 0.6, 1.0, -0.5, 0.5, 10]
Set 15: [14, 30, 80, 0.3, 0.762, 0.536, 0.4, 12]
Set 16: [14, 33, 82, 0.476, 0.6, -0.4, 1.21, 8]
Set 17: [12, 32, 80, 0.476, 1.0, -0.4, 0.5, 6]
Set 18: [12, 31, 82, 0.3, 0.9, 0.536, 0.3, 10]
Set 19: [12, 30, 83, 0.6, 0.6, -0.5, 1.21, 12]
Set 20: [12, 33, 81, 0.7, 0.762, -0.3, 0.4, 8]
Set 21: [18, 32, 80, 0.3, 0.9, -0.5, 1.21, 8]
Set 22: [18, 31, 82, 0.

In [14]:
in_val_sets = np.array(set_list)
in_val_sets[:, 0] = in_val_sets[:, 0] / 2 # divide by 2 to get z_sh

In [ ]:
%load_ext autoreload
%autoreload 2
from utils import validate_parameters

In [16]:
z_r1 = 90
z_p1 = 39
z_s = 12
z_p2 = 32
z_r2 = 81
xr1 = 2
xr2 = 1.2
xs = 0.476
xp1 = 0.762
I1 = z_r1 / z_s
I2 = (z_r1 * z_p2) / (z_p1 * z_r2)
gr_s = (1 - I2) / (1 + I1)
ratios = 1 / gr_s

print(I1, I2, gr_s, ratios)

7.5 0.9116809116809117 0.01039048097871627 96.24193548387099


In [ ]:
((z_r1+2*xr1-z_s-2*xs-4*xp1)/2)

39.0

In [18]:
validate_parameters(in_val_sets[0][:-1])

ValueError: too many values to unpack (expected 5)

In [24]:
this_set = in_val_sets[0]
ra = 0.5 * MODULE * ( this_set[0]*2 + 2*this_set[3] +  

SyntaxError: incomplete input (3891727407.py, line 2)

In [10]:
validations = []
for i in range(len(in_val_sets)):
    validations.append(np.is_empty(validate_parameters(in_val_sets[i][:-1])))

AttributeError: module 'numpy' has no attribute 'is_empty'

In [ ]:
validations

[]